<a href="https://colab.research.google.com/github/etomoscow/Complete-Python-3-Bootcamp/blob/master/Lab2_DL_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 3

### Part 1. Overfit it (1.5 points)

Будем работать с датасетом [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) (*hint: он доступен в torchvision*).

Ваша задача состоит в следующем:
1. Обучить сеть, которая покажет >= 0.92 test accuracy.
2. Пронаблюдать и продемонстрировать процесс переобучения сети с увеличением числа параметров (==нейронов) и/или числа слоев и продемонстрировать это наглядно (например, на графиках).
3. Попробовать частично справиться с переобучением с помощью подходящих приемов (Dropout/batchnorm/augmentation etc.)

*Примечание*: Пункты 2 и 3 взаимосвязаны, в п.3 Вам прелагается сделать полученную в п.2 сеть менее склонной к переобучению. Пункт 1 является независимым от пунктов 2 и 3.

# Part 1

In [0]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt 
import numpy as np 
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import helper
import math

In [0]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
# Download and load the training data
trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1)

# Download and load the test data
testset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1)

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=10):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Linear(256 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = ResNet(BasicBlock, [2, 2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters())

In [10]:
num_epochs = 15
count = 0
# Lists for visualization of loss and accuracy 
loss_list = []
iteration_list = []
accuracy_list = []

# Lists for knowing classwise accuracy
predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfering images and labels to GPU if available
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(1, 1, 28, 28))
        labels = Variable(labels)
        
        # Forward pass 
        outputs = model(train)
        loss = criterion(outputs, labels)
        
        # Initializing a gradient as 0 so there is no mixing of gradient among the batches
        optimizer.zero_grad()
        #Propagating the error backward
        loss.backward()
        # Optimizing the parameters
        optimizer.step()
    
        count += 1
    
    # Testing the model
    
        if not (count % 50):
            total = 0
            correct = 0
        
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
            
                test = Variable(images.view(1, 1, 28, 28))
            
                outputs = model(test)
            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

Iteration: 500, Loss: 0.8784523010253906, Accuracy: 65%
Iteration: 1000, Loss: 0.1266353726387024, Accuracy: 71%
Iteration: 1500, Loss: 0.06967642158269882, Accuracy: 73%


KeyboardInterrupt: ignored

In [0]:
class_correct = [0. for _ in range(10)]
total_correct = [0. for _ in range(10)]

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        test = Variable(images)
        outputs = model(test)
        predicted = torch.max(outputs, 1)[1]
        c = (predicted == labels).squeeze()
        
        for i in range(100):
            label = labels[i]
            class_correct[label] += c[i].item()
            total_correct[label] += 1
        
for i in range(10):
    print("Accuracy of class {}: {:.2f}%".format(i, class_correct[i] * 100 / total_correct[i]))

# Part 2

#### Простая сеть только из линейных слоёв - отличный способ показать проблему переобучения

In [0]:
class OverfitNet(nn.Module):  
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(784, 256)
    self.l2 = nn.Linear(256, 128)
    self.l3 = nn.Linear(128, 64)
    self.l4 = nn.Linear(64, 10)

  def forward(self):
    #сглаживание входного тензора, чтобы избежать несовпадений размерностей
    x = x.view(x.shape[0], -1)

    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    
    x = F.softmax(x, dim=1)
    
    return x

In [0]:
of_model  = OverfitNet().to(device)

#loss
criterion = nn.NLLLoss()

#функция оптимизации
opt = torch.optim.Adam(of_model.parameters(), lr=3e-3)

num_epochs = 30

train_losses = []
test_losses = []

for i in range(num_epochs):
  run_loss = 0
  
  for images,labels in trainloader:
    '''обнуляем градиент, иначе, так как у нас chain rule в вычислении градиента,
    то через пару итераций мы просто забьём всю RAM ненужными цифрами'''
    optimizer.zero_grad()              
    '''подаём изображение на вход сети (forward pass по сути дела)'''
    log_ps = model(images)                     
    '''вычисляем loss'''
    loss = criterion(log_ps, labels)       
    '''backward pass'''
    loss.backward()                            
    optimizer.step()                           
    run_loss += loss.item()                
     
    
  test_loss = 0
  accuracy = 0
    
  # Turn off the gradients
  with torch.no_grad():
    # цикл по валидационному датасету
    for images, labels in testloader:
      log_ps = model(images)                                 
      ps = torch.exp(log_ps)                                 
      test_loss += criterion(log_ps, labels)             
      top_p, top_class = ps.topk(1,dim=1)                    
      equals = top_class == labels.view(*top_class.shape)   
      accuracy += torch.mean(equals.type(torch.FloatTensor))
    
  # записываем средние лоссы на трейне и тесте       
  train_losses.append(run_loss/len(trainloader))
  test_losses.append(test_loss/len(testloader)) 

  #выводим промежуточные результаты
  print("Epoch: {}/{}.. ".format(i+1, num_epochs),
              "Training Loss: {:.3f} ".format(run_loss/len(trainloader)),
              "Test Loss: {:.3f} ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

#### Посмотрим на лоссы.

В идеале две кривые должны вести себя примерно одинаково - постепенно снижаться как, например, график $y = \frac{1}{x}.$

In [0]:
plt.figure(figsize=(8,8))
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend(loc='best')
plt.show()

#### Как видно, этого не происходит - на тесте лосс колеблется в четко выделяемом интервале и не снижается. Это и есть переобучение в нейронных сетях. 

Что это значит? Это значит, что сеть не показывает хороший результат вне тренировочного датасета. 

Попробуем избавиться от переобучения, добавив `Dropout`

Даже один `Dropout` на некотором этапе обучения сети может существенно улучшить её работу на инференсе. Например, рассмотрим такую сеть:

In [0]:
class OverfitNet_Dropout(nn.Module):  
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(784, 256)
    self.l2 = nn.Linear(256, 128)
    self.l3 = nn.Linear(128, 64)
    self.l4 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.2)

  def forward(self):
    #сглаживание входного тензора, чтобы избежать несовпадений размерностей
    x = x.view(x.shape[0], -1)

    x = self.do(F.relu(self.l1(x)))
    x = self.do(F.relu(self.l2(x)))
    x = self.do(F.relu(self.l3(x)))
    x = self.do(F.relu(self.l4(x)))
    
    x = F.softmax(x, dim=1)
    
    return x

Снова обучим сеть и посмотрим на графики: 

In [0]:
of_model  = OverfitNet_Dropout().to(device)

#loss
criterion = nn.NLLLoss()

#функция оптимизации
opt = torch.optim.Adam(of_model.parameters(), lr=3e-3)

num_epochs = 30

train_losses = []
test_losses = []

for i in range(num_epochs):
  run_loss = 0
  
  for images,labels in trainloader:
    '''обнуляем градиент, иначе, так как у нас chain rule в вычислении градиента,
    то через пару итераций мы просто забьём всю RAM ненужными цифрами'''
    optimizer.zero_grad()              
    '''подаём изображение на вход сети (forward pass по сути дела)'''
    log_ps = model(images)                     
    '''вычисляем loss'''
    loss = criterion(log_ps, labels)       
    '''backward pass'''
    loss.backward()                            
    optimizer.step()                           
    run_loss += loss.item()                
     
    
  test_loss = 0
  accuracy = 0
    
  # Turn off the gradients
  with torch.no_grad():
    # цикл по валидационному датасету
    for images, labels in testloader:
      log_ps = model(images)                                 
      ps = torch.exp(log_ps)                                 
      test_loss += criterion(log_ps, labels)             
      top_p, top_class = ps.topk(1,dim=1)                    
      equals = top_class == labels.view(*top_class.shape)   
      accuracy += torch.mean(equals.type(torch.FloatTensor))
    
  # записываем средние лоссы на трейне и тесте       
  train_losses.append(run_loss/len(trainloader))
  test_losses.append(test_loss/len(testloader)) 

  #выводим промежуточные результаты
  print("Epoch: {}/{}.. ".format(i+1, num_epochs),
              "Training Loss: {:.3f} ".format(run_loss/len(trainloader)),
              "Test Loss: {:.3f} ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

In [0]:
plt.figure(figsize=(8,8))
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend(loc='best')
plt.show()